<a href="https://colab.research.google.com/github/mashyko/NLP_BERT_Transformers/blob/master/Mecab_Neologd_livedoor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MecabとGensim による類似単語検出：Livedoor News を使用

Mecabとmecab-ipadic-neologdのインストール： 

NEologd辞書の追加　:
NEologdは、Web上から得た新語に対応しており、毎週更新されるMeCab用のシステム辞書です。

mecab-python もインストール


In [0]:
!apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
!pip install mecab-python3 > /dev/null

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.


mecab-ipadic-neologdが保存されているフォルダーを調べる。

In [0]:
!mecab-config --dicdir

/usr/lib/x86_64-linux-gnu/mecab/dic


In [0]:
!cat /etc/mecabrc

;
; Configuration file of MeCab
;
; $Id: mecabrc.in,v 1.3 2006/05/29 15:36:08 taku-ku Exp $;
;
dicdir = /var/lib/mecab/dic/debian

; userdic = /home/foo/bar/user.dic

; output-format-type = wakati
; input-buffer-size = 8192

; node-format = %m\n
; bos-format = %S\n
; eos-format = EOS\n



In [0]:
import MeCab
sent = "スマホでインスタが流行っている" 
tagger = MeCab.Tagger ()
print(tagger.parse(sent))

スマホ	名詞,一般,*,*,*,*,*
で	助詞,格助詞,一般,*,*,*,で,デ,デ
インスタ	名詞,一般,*,*,*,*,*
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
流行っ	動詞,自立,*,*,五段・ラ行,連用タ接続,流行る,ハヤッ,ハヤッ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
いる	動詞,非自立,*,*,一段,基本形,いる,イル,イル
EOS



In [0]:
!echo "SMAPのキムタクはポケモンに会いに行き、スマホを売るバイトをしている" | mecab -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd

SMAP	名詞,固有名詞,一般,*,*,*,SMAP,スマップ,スマップ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
キムタク	名詞,固有名詞,一般,*,*,*,キムタク,キムタク,キムタク
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
ポケモン	名詞,固有名詞,一般,*,*,*,Pokemon!,ポケモン,ポケモン
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
会い	名詞,一般,*,*,*,*,会意,カイイ,カイー
に	助詞,格助詞,一般,*,*,*,に,ニ,ニ
行き	動詞,自立,*,*,五段・カ行促音便,連用形,行く,イキ,イキ
、	記号,読点,*,*,*,*,、,、,、
スマホ	名詞,固有名詞,一般,*,*,*,スマホ,スマホ,スマホ
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
売る	動詞,自立,*,*,五段・ラ行,基本形,売る,ウル,ウル
バイト	名詞,一般,*,*,*,*,バイト,バイト,バイト
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
し	動詞,自立,*,*,サ変・スル,連用形,する,シ,シ
て	助詞,接続助詞,*,*,*,*,て,テ,テ
いる	動詞,非自立,*,*,一段,基本形,いる,イル,イル
EOS


ここからLivedoor News を使用しますので、以下のようにファイルをダウンロードして配置してください。

/My Drive/NLP/livedoor-news-data/topic-news.xml 

この文書が入ったMy Drive にmountします。

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#!mecab -Owakati topic-news.txt -o topic-wakati.txt
!mecab  -Owakati /content/drive/My\ Drive/NLP/livedoor-news-data/topic-news.txt -o /content/drive/My\ Drive/NLP/livedoor-news-data/topic-wakati.txt

In [0]:
import MeCab
tagger = MeCab.Tagger('-F\s%f[6] -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')

raw = open('/content/drive/My Drive/NLP/livedoor-news-data/topic-news.xml', 'r')
text = open('/content/drive/My Drive/NLP/livedoor-news-data/topic-news_text.xml', 'w')

line = raw.readline()
while line:
    result = tagger.parse(line)
    text.write(result[1:]) # skip first \s
    line = raw.readline()

raw.close()
text.close()

In [0]:
%%bash
cd /content/drive/My\ Drive/NLP/livedoor-news-data/
ls 

dokujo-tsushin.xml
it-life-hack.xml
kaden-channel.xml
livedoor-homme.xml
movie-enter.xml
peachy.xml
smax.xml
sports-watch.xml
topic-news.model
topic-news_text.txt
topic-news_text.xml
topic-news.txt
topic-news.xml
topic-wakati.txt


Gensimのインストール

In [0]:
!pip install gensim

In [0]:
from gensim.models import word2vec
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

sentences = word2vec.LineSentence('/content/drive/My Drive/NLP/livedoor-news-data/topic-news_text.xml')
model = word2vec.Word2Vec(sentences,
                          sg=1,
                          size=100,
                          min_count=1,
                          window=10,
                          hs=1,
                          negative=0)
model.save('/content/drive/My Drive/NLP/livedoor-news-data/topic-news.model')


2019-12-29 04:46:59,913 : INFO : collecting all words and their counts
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2019-12-29 04:46:59,926 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-12-29 04:47:00,010 : INFO : PROGRESS: at sentence #10000, processed 215902 words, keeping 14007 word types
2019-12-29 04:47:00,072 : INFO : collected 19336 word types from a corpus of 377784 raw words and 16734 sentences
2019-12-29 04:47:00,073 : INFO : Loading a fresh vocabulary
2019-12-29 04:47:00,173 : INFO : effective_min_count=1 retains 19336 unique words (100% of original 19336, drops 0)
2019-12-29 04:47:00,174 : INFO : effective_min_count=1 leaves 377784 word 

In [0]:
model = word2vec.Word2Vec.load('/content/drive/My Drive/NLP/livedoor-news-data/topic-news.model')
model.wv.most_similar(positive=['嵐'],topn=20)

2019-12-29 04:47:14,572 : INFO : loading Word2Vec object from /content/drive/My Drive/NLP/livedoor-news-data/topic-news.model
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2019-12-29 04:47:14,867 : INFO : loading wv recursively from /content/drive/My Drive/NLP/livedoor-news-data/topic-news.model.wv.* with mmap=None
2019-12-29 04:47:14,868 : INFO : setting ignored attribute vectors_norm to None
2019-12-29 04:47:14,877 : INFO : loading vocabulary recursively from /content/drive/My Drive/NLP/livedoor-news-data/topic-news.model.vocabulary.* with mmap=None
2019-12-29 04:47:14,881 : INFO : loading trainables recursively from /content/drive/My Drive/NLP/livedoor-news-data/topic-news.model.t

[('生歌', 0.7207505702972412),
 ('酷', 0.6602052450180054),
 ('機材', 0.6535590291023254),
 ('テレビ放送', 0.6445494890213013),
 ('櫻井翔', 0.6333144903182983),
 ('FNS歌謡祭', 0.6314767003059387),
 ('連ドラ', 0.6109600067138672),
 ('新', 0.6080925464630127),
 ('反対デモ', 0.6029863357543945),
 ('ディー・エヌ・エー', 0.5936254262924194),
 ('女子アナ', 0.5878632068634033),
 ('騒然', 0.5859134197235107),
 ('大島優子', 0.585310697555542),
 ('吉高由里子', 0.5827311277389526),
 ('タカトシ', 0.5761194229125977),
 ('こわい', 0.5748745203018188),
 ('特集', 0.5724114179611206),
 ('丸見え', 0.5715619325637817),
 ('SKE', 0.5681651830673218),
 ('した', 0.5663911700248718)]

以上